In [1]:
import pandas as pd
from sklearn.externals import joblib
import numpy as np
import talib
from sklearn.preprocessing import MinMaxScaler

In [2]:
def addTechnicalFeatures(df):
    df['Data'] = pd.to_datetime(df['Data'])
    df.columns = ['Data', 'Open','High', 'Low', 'Close', 'Wol', "Lop"]
    #Wskazniki analizy technicznej
    k, dfast = talib.STOCH(np.array(df['High']),np.array(df['Low']),
                           np.array(df['Close'])) # uses high, low, close by default
    df['k'] = k
    df['dfast'] = dfast
    df['dslow'] = talib.SMA(dfast, timeperiod=5)
    df['momentum'] = talib.MOM(np.array(df['Close']), timeperiod=4)
    df['roc'] = talib.ROC(np.array(df['Close']), timeperiod=5)
    df['willR'] = talib.WILLR(np.array(df['High']), np.array(df['Low']),
                        np.array(df['Close']), timeperiod = 5)
    #ad = talib.ADOSC(np.array(df['High']), np.array(df['Low']),
    #                          np.array(df['Close']), np.array(df['Wol']))
    df['disp5'] = df['Close'] / talib.SMA(np.array(df['Close']), 5) * 100
    df['disp10'] = df['Close'] / talib.SMA(np.array(df['Close']), 10) * 100
    df['oscp'] = ((talib.SMA(np.array(df['Close']), 5) - talib.SMA(np.array(df['Close']), 10)) / 
                                                        talib.SMA(np.array(df['Close']), 5)) 
    df['rsi'] = talib.RSI(np.array(df['Close']))
    df['CCI'] = talib.CCI(np.array(df['High']),np.array(df['Low']), np.array(df['Close']))
    #Tworzenie zmiennej celu
    df['target1'] = df['Close'].shift(-1) -df['Open']
    df['target5'] = df['Close'].shift(-5) -df['Open']
    df['target3'] = df['Close'].shift(-3) -df['Open']
    #zostawienie tylko zmiennej celu i wskaźników technicznych
    df.drop(['Data','Open', 'High', 'Low', 'Close', 'Wol', 'Lop'],axis=1, inplace=True)
    return df

In [3]:
rfSPY = joblib.load("RandomForestSPY.pkl")

In [4]:
df = pd.read_csv("https://stooq.pl/q/d/l/?s=es.f&i=d")

In [6]:
df = addTechnicalFeatures(df)

In [7]:
X = df.drop(['target1','target3','target5'], axis=1)
X.dropna(inplace=True, axis=0, how="any")

In [8]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

Wchodzimy gdy jest to wieksze niz 6.59, na 5 dni

In [16]:
rfSPY.predict(X[-1:,])

array([ 1.00890004])

In [17]:
df = pd.read_csv("https://stooq.pl/q/d/l/?s=nq.f&i=d")

In [18]:
df = addTechnicalFeatures(df)

In [19]:
X = df.drop(['target1','target3','target5'], axis=1)
X.dropna(inplace=True, axis=0, how="any")
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [20]:
rfNDQ = joblib.load("randomForestNasdaq.pkl")

Wchodzimy, gdy jest to większy niż 19.64

In [21]:
rfNDQ.predict(X[-1:,:])

array([ 15.29184004])

In [22]:
df  = pd.read_csv("https://stooq.pl/q/d/l/?s=ym.f&i=d")

In [23]:
df = addTechnicalFeatures(df)

In [24]:
X = df.drop(['target1','target3','target5'], axis=1)
X.dropna(inplace=True, axis=0, how="any")
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [25]:
rfDIJ = joblib.load("randomForestDIJA.pkl")

Tutaj wchodzimy gdy jest wieksze niz 63

In [27]:
rfDIJ.predict(X[-1:,:])

array([ 49.07760205])